In [1]:
import pandas as pd
import numpy as np
# For cleaning the text
import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import regex as re
import string
import pandas as pd
import numpy as np
import nltk.data
import re
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, sent_tokenize, pos_tag

[nltk_data] Downloading package punkt to
[nltk_data]     /home/catherinesai/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/catherinesai/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/catherinesai/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/catherinesai/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
from file_paths import *
df_reg = pd.read_excel(join(INTERMEDIATE_DIRECTORY, "gdpr_reg_relevant_sentences.xlsx"), index_col=0, names=['text']) 
df_rea = pd.read_excel(join(INTERMEDIATE_DIRECTORY, "gdpr_rea_relevant_sentences.xlsx"), index_col=0, names=['text']) 

In [10]:
df_reg.head()

,text
0,A transfer of personal data to a third country...
1,Such a transfer shall not require any specific...
2,When assessing the adequacy of the level of pr...
3,When assessing the adequacy of the level of pr...
4,When assessing the adequacy of the level of pr...


In [11]:
df_rea.head()

,text
0,The processor can process personal data only a...
1,Processors may engage other controller or thir...
2,the controller's prior consent will be granted...
3,the controller's prior consent must also oblig...
4,The form of consent as well as information obl...


In [12]:
def preprocess_text(text):
    # 1. Tokenise to alphabetic tokens
    text = remove_numbers(text)
    text = remove_http(text)
    text = remove_punctuation(text)
    text = convert_to_lower(text)
    text = remove_white_space(text)
    text = remove_short_words(text)
    tokens = toknizing(text)
    # 2. POS tagging
    pos_map = {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'}
    pos_tags_list = pos_tag(tokens)
    #print(pos_tags)
    # 3. Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    tokens = [lemmatiser.lemmatize(w.lower(), pos=pos_map.get(p[0], 'v')) for w, p in pos_tags_list]
    return tokens
#------------------------------------------------------------------------------
def convert_to_lower(text):
  return text.lower()
#------------------------------------------------------------------------------
def remove_numbers(text):
  text = re.sub(r'\d+' , '', text)
  return text
#------------------------------------------------------------------------------
def remove_http(text):
  text = re.sub("https?:\/\/t.co\/[A-Za-z0-9]*", ' ', text)
  return text
#------------------------------------------------------------------------------
def remove_short_words(text):
  text = re.sub(r'\b\w{1,2}\b', '', text)
  return text
#------------------------------------------------------------------------------
def remove_punctuation(text):
     punctuations = '''!()[]{};«№»:'"\,`<>./?@=#$-(%^)+&[*_]~'''
     no_punct = ""
     
     for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
     return no_punct
#------------------------------------------------------------------------------
def remove_white_space(text):
  text = text.strip()
  return text
#------------------------------------------------------------------------------
def toknizing(text):
  stop_words = set(stopwords.words('english'))
  tokens = word_tokenize(text)
  ## Remove Stopwords from tokens
  result = [i for i in tokens if not i in stop_words]
  return result

In [13]:
#clean text applying all the text preprocessing functions
df_reg['cleaned_text'] = df_reg.text.apply(lambda x: ' '.join(preprocess_text(x)))
df_reg.head()

,text,cleaned_text
0,A transfer of personal data to a third country...,transfer personal data third country internati...
1,Such a transfer shall not require any specific...,transfer shall require specific authorisation
2,When assessing the adequacy of the level of pr...,assess adequacy level protection commission sh...
3,When assessing the adequacy of the level of pr...,assess adequacy level protection commission sh...
4,When assessing the adequacy of the level of pr...,assess adequacy level protection commission sh...


In [14]:
#clean text applying all the text preprocessing functions
df_rea['cleaned_text'] = df_rea.text.apply(lambda x: ' '.join(preprocess_text(x)))
df_rea.head()

,text,cleaned_text
0,The processor can process personal data only a...,processor process personal data per controller...
1,Processors may engage other controller or thir...,processor may engage controller third party su...
2,the controller's prior consent will be granted...,controller prior consent grant processor subje...
3,the controller's prior consent must also oblig...,controller prior consent must also oblige subc...
4,The form of consent as well as information obl...,form consent well information obligation event...


In [43]:
import yake

def key_word_extractor (text):
    language = "en"
    max_ngram_size = 3
    deduplication_thresold = 0.9
    deduplication_algo = 'seqm'
    windowSize = 1
    numOfKeywords = 2
    kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords)
    keywords = kw_extractor.extract_keywords(text)
    kw_list = [] 
    for kw, v in keywords:
        kw_list.append(kw)
    return kw_list

In [46]:
df_reg['key_words'] = df_reg.cleaned_text.apply(lambda x: key_word_extractor(x))
df_reg['key_words'] = df_reg.key_words.apply(lambda x: ' '.join(x))
pd.set_option('display.max_colwidth', None)
df_reg.head()

,text,cleaned_text,key_words
0,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",transfer personal data third country international organisation may take place commission decide third country territory one specify sector within third country international organisation question ensure adequate level protection,country international organisation transfer personal data
1,Such a transfer shall not require any specific authorisation.,transfer shall require specific authorisation,require specific authorisation transfer shall require
2,"When assessing the adequacy of the level of protection, the Commission shall, in particular, take account of the rule of law, respect for human rights and fundamental freedoms, relevant legislation, both general and sectoral, including concerning public security, defence, national security and criminal law and the access of public authorities to personal data, as well as the implementation of such legislation, data protection rules, professional rules and security measures, including rules for the onward transfer of personal data to another third country or international organisation which are complied with in that country or international organisation, case-law, as well as effective and enforceable data subject rights and effective administrative and judicial redress for the data subjects whose personal data are being transferred.",assess adequacy level protection commission shall particular take account rule law respect human right fundamental freedom relevant legislation general sectoral include concern public security defence national security criminal law access public authority personal data well implementation legislation data protection rule professional rule security measure include rule onward transfer personal data another third country international organisation comply country international organisation caselaw well effective enforceable data subject right effective administrative judicial redress data subject whose personal data transfer,country international organisation assess adequacy level
3,"When assessing the adequacy of the level of protection, the Commission shall, in particular, take account of the existence and effective functioning of one or more independent supervisory authorities in the third country or to which an international organisation is subject, with responsibility for ensuring and enforcing compliance with the data protection rules, including adequate enforcement powers, for assisting and advising the data subjects in exercising the data protection rules, including adequate enforcement powers rights and for cooperation with the supervisory authorities of the Member States.",assess adequacy level protection commission shall particular take account existence effective function one independent supervisory authority third country international organisation subject responsibility ensure enforce compliance data protection rule include adequate enforcement power assist advise data subject exercise data protection rule include adequate enforcement power right cooperation supervisory authority member state,rule include adequate include adequate enforcement
4,"When assessing the adequacy of the level of protection, the Commission shall, in particular, take account of the international commitments the third country or international organisation concerned has entered into, or other obligations arising from legally binding conventions or instruments as well as from the Commission participation in multilateral or regional systems, in particular in relation to the protection of personal data.",assess adequacy level protection commission shall particular take account international commitment third country international organ

In [47]:
df_rea['key_words'] = df_rea.cleaned_text.apply(lambda x: key_word_extractor(x))
df_rea['key_words'] = df_rea.key_words.apply(lambda x: ' '.join(x))
pd.set_option('display.max_colwidth', None)
df_rea.head()

,text,cleaned_text,key_words
0,The processor can process personal data only as per the controller's instructions.,processor process personal data per controller instruction,processor process personal process personal data
1,"Processors may engage other controller or third parties (""subcontractors"") to process personal data in Processors own (sub) contract only with the controller's prior consent.",processor may engage controller third party subcontractor process personal data processor sub contract controller prior consent,controller prior consent party subcontractor process
2,the controller's prior consent will be granted only if The processor subjects the subcontractor – contractually or by other comparable legally binding means – to the same data protection obligations to which The processor is subject pursuant to this policy vis-a-vis the controller and data subjects.,controller prior consent grant processor subject subcontractor – contractually comparable legally bind mean – data protection obligation processor subject pursuant policy visavis controller data subject,contractually comparable legally prior consent grant
3,the controller's prior consent must also oblige the subcontractor to take the appropriate technical and organizational protective measures.,controller prior consent must also oblige subcontractor take appropriate technical organizational protective measure,controller prior consent organizational protective measure
4,The form of consent as well as information obligations in the event of changes in the subcontracted relationship must be set out in the contract for services.,form consent well information obligation event change subcontract relationship must set contract service,set contract service information obligation event


In [48]:
from itertools import product
result_df = pd.DataFrame()
# rename columns
df_reg = df_reg.rename(columns={"text": "original_sentence_reg", "cleaned_text": "cleaned_text_reg", "key_words": "key_words_reg"})
df_rea = df_rea.rename(columns={"text": "original_sentence_rea", "cleaned_text": "cleaned_text_rea", "key_words": "key_words_rea"})
data = list(product(df_reg['original_sentence_reg'], df_rea['original_sentence_rea']))
result_df = result_df.append(data)
result_df = result_df.rename(columns={0:'original_sentence_reg',1:'original_sentence_rea'})
result_df = result_df.reset_index(drop=True)
result_df.head()

,original_sentence_reg,original_sentence_rea
0,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",The processor can process personal data only as per the controller's instructions.
1,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.","Processors may engage other controller or third parties (""subcontractors"") to process personal data in Processors own (sub) contract only with the controller's prior consent."
2,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",the controller's prior consent will be granted only if The processor subjects the subcontractor – contractually or by other comparable legally binding means – to the same data protection obligations to which The processor is subject pursuant to this policy vis-a-vis the controller and data subjects.
3,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",the controller's prior consent must also oblige the subcontractor to take the appropriate technical and organizational protective measures.
4,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",The form of consent as well as information obligations in the event of changes in the subcontracted relationship must be set out in the contract for services.


In [49]:
result_df = pd.merge(result_df, df_reg,  how='left', left_on=['original_sentence_reg'], right_on = ['original_sentence_reg'])
result_df = pd.merge(result_df, df_rea,  how='left', left_on=['original_sentence_rea'], right_on = ['original_sentence_rea'])
result_df.head()

,original_sentence_reg,original_sentence_rea,cleaned_text_reg,key_words_reg,cleaned_text_rea,key_words_rea
0,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",The processor can process personal data only as per the controller's instructions.,transfer personal data third country international organisation may take place commission decide third country territory one specify sector within third country international organisation question ensure adequate level protection,country international organisation transfer personal data,processor process personal data per controller instruction,processor process personal process personal data
1,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.","Processors may engage other controller or third parties (""subcontractors"") to process personal data in Processors own (sub) contract only with the controller's prior consent.",transfer personal data third country international organisation may take place commission decide third country territory one specify sector within third country international organisation question ensure adequate level protection,country international organisation transfer personal data,processor may engage controller third party subcontractor process personal data processor sub contract controller prior consent,controller prior consent party subcontractor process
2,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",the controller's prior consent will be granted only if The processor subjects the subcontractor – contractually or by other comparable legally binding means – to the same data protection obligations to which The processor is subject pursuant to this policy vis-a-vis the controller and data subjects.,transfer personal data third country international organisation may take place commission decide third country territory one specify sector within third country international organisation question ensure adequate level protection,country international organisation transfer personal data,controller prior consent grant processor subject subcontractor – contractually comparable legally bind mean – data protection obligation processor subject pursuant policy visavis controller data subject,contractually comparable legally prior consent grant
3,"A transfer of personal data to a third country or an international organisation may take place where the Commission has decided that the third country, a territory or one or more specified sectors within that third country, or the international organisation in question ensures an adequate level of protection.",the controller's prior consent must also oblige the subcontractor to take the appropriate technical and organizational protective measures.,transfer personal data third country international organisation may take place commission decide third country territory one specify sector within third country international organisation question ensure adequate level protection,country international organisation transfer personal data,controller prior consent must also oblige subcontractor take appropriate technical organizational protective measure,controller prior consent organizational protective measure
4,"A transfer of personal data to a third country or an i

In [51]:
nlp = spacy.load('en_core_web_lg')
nlp.add_pipe("merge_entities")

<function spacy.pipeline.functions.merge_entities(doc: spacy.tokens.doc.Doc)>

In [52]:
threshold = 0.67
for index, row in result_df.iterrows():
    doc_1 = nlp(row['key_words_reg'])
    doc_2 = nlp(row['key_words_rea'])
    #doc_1 = self.nlp(row['original_sentence_reg'])
    #doc_2 = self.nlp(row['original_sentence_rea'])
    result_df.at[index, 'similarity'] = doc_1.similarity(doc_2)
    # group by reg sentences to get one row per reg sentence; per reg sentence keep row with max similarity score 
    res = result_df[result_df['similarity'] == result_df.groupby('original_sentence_reg')['similarity'].transform('max')]
    # extract the matched sentence-pairs -> for further in depth analysis
    reg_rea_sentence_matches_df = res[res['similarity'] >= threshold]

ValueError: No engine for filetype: ''

In [53]:
reg_rea_sentence_matches_df .to_excel(join(INTERMEDIATE_DIRECTORY, 'key_phrases.xlsx'))  

In [15]:
def get_tokened_docs(dtf):
  doclist= []
  for d in dtf['cleaned_text']:
    doclist.append(d)
  return doclist

In [16]:
docs_reg = get_tokened_docs(df_reg)
docs_rea = get_tokened_docs(df_rea)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True, max_df=0.3, min_df=1, ngram_range=(1,3))
vectors = vectorizer.fit_transform(df_reg['cleaned_text'])
# get the token list
fnames = vectorizer.get_feature_names()
dict_of_tokens={i[1]:i[0] for i in vectorizer.vocabulary_.items()}
tfidf_vectors = []  # all deoc vectors by tfidf
for row in vectors:
  tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})
# list of doc features each with tfidf weight
doc_sorted_tfidfs =[]  
#sort each dict of a document
for dn in tfidf_vectors:
  newD = sorted(dn.items(), key=lambda x: x[1], reverse=True)
  newD = dict(newD)
  doc_sorted_tfidfs.append(newD)
tfidf_kw = []
for doc_tfidf in doc_sorted_tfidfs:
  ll = list(doc_tfidf.keys())
  tfidf_kw.append(ll)
TopN= 5
print(tfidf_kw[0][0:TopN])

['third country', 'country', 'third', 'country international organisation', 'third country international']
